# Pretrained Model + Tfidf + KNN Clustering for Image Prediction

In [21]:
import numpy as np 
import pandas as pd 

#reading data
import pickle

#Data visutalisation 
import matplotlib.pyplot as plt 
import seaborn as sns 

#Tensorflow, teras 
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv2D, Concatenate, Input
from tensorflow.keras.layers import GlobalAveragePooling2D, GlobalMaxPooling2D, MaxPool2D,AveragePooling2D,MaxPooling2D,BatchNormalization
from sklearn.metrics import  accuracy_score as score
import tensorflow as tf
from sklearn.model_selection import train_test_split
import keras
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from keras.utils import to_categorical

#Text Cleaning
import sklearn.feature_extraction.text as text
from nltk.corpus import stopwords
from  nltk.stem import SnowballStemmer
import matplotlib.pyplot as plt 
from sklearn import preprocessing
from nltk.stem import WordNetLemmatizer 
import re

#Keras
import keras
from keras.preprocessing import image
from keras.applications.imagenet_utils import decode_predictions, preprocess_input
from keras.models import Model
from tqdm.notebook import tqdm as tqdm
from tensorflow.keras.applications import EfficientNetB0

#Clustering
from sklearn.neighbors import NearestNeighbors

#Dim reduction
from sklearn.decomposition import PCA

To start, let's load the Kaggle dataset to Pandas.

In [22]:
# You will need to change the path.
df = pd.read_csv(r'C:\Users\David\Documents\code\Mod_5\5-4-modelling-competition\shopee-product-matching\train.csv')

Text cleaning functions and Tfidf.

In [23]:
# Cleaning the data  for the titles 

TEXT_CLEANING_RE = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"

# to remove the stop words
stop_words = stopwords.words("english")

stemmer = SnowballStemmer("english")
lemmatizer = WordNetLemmatizer() 

#preprocess function 
def preprocess(text, stem=False,lem=True):
    # Remove link and special characters
    text = re.sub(TEXT_CLEANING_RE, ' ', str(text).lower()).strip()
    tokens = []
    for token in text.split():
        if token not in stop_words:
            if stem:
                tokens.append(stemmer.stem(token))
            if lem:
                tokens.append(lemmatizer.lemmatize(token))
            else:
                tokens.append(token)
    return " ".join(tokens)

# cleaning the data for every rows
df.title= df.title.apply(lambda x: preprocess(x))

# TF-Idf
#---------------------------------------------------
tfidf = text.TfidfVectorizer(min_df=2) #ok 


tfidf.fit(df.title)
t = tfidf.transform(df.title)

#just making a matrix.
matrice=pd.DataFrame(t.todense())
matrice.columns=tfidf.get_feature_names()

#and to an array.
t=(t.todense())
#---------------------------------------------------

Load the pretrained model (VGG16).

In [24]:
#Change the path
WGT=r'C:\Users\David\Documents\code\Mod_5\5-4-modelling-competition\efficientnetb0.h5'

#this pretrained model accepts imputs with shape (224,224,3)
pretrained_model = EfficientNetB0(weights=WGT, include_top=True)

#the output of the cv2 layer has the shape (1, 4096)
feat_extractor = Model(inputs=pretrained_model.input, outputs=pretrained_model.get_layer('top_dropout').output)

#summary of the feature extractor 
feat_extractor.summary()

t_conv[0][0]       
__________________________________________________________________________________________________
block5b_drop (Dropout)          (None, 14, 14, 112)  0           block5b_project_bn[0][0]         
__________________________________________________________________________________________________
block5b_add (Add)               (None, 14, 14, 112)  0           block5b_drop[0][0]               
                                                                 block5a_project_bn[0][0]         
__________________________________________________________________________________________________
block5c_expand_conv (Conv2D)    (None, 14, 14, 672)  75264       block5b_add[0][0]                
__________________________________________________________________________________________________
block5c_expand_bn (BatchNormali (None, 14, 14, 672)  2688        block5c_expand_conv[0][0]        
_________________________________________________________________________________________

In [25]:
#shape of the loaded image for the pretrained model 
im_input_shape = feat_extractor.output.shape[1:]

#the shape is the number of columns of he TF-IDF matrix
tfidf_input_shape = t.shape[1]

Implement simple dense layer for tfidf training.

In [26]:
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv2D, Concatenate

x_im_feat = Input(shape=im_input_shape, name="Images-data")
tfifd_input = Input(shape=tfidf_input_shape, name="tf-idf-data")

x_Tf_idf = Dense(1280, activation='relu')(tfifd_input)

x=x_im_feat*x_Tf_idf

model = Model(inputs=[x_im_feat,tfifd_input], outputs=x, name='test')
model.summary()

Model: "test"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
tf-idf-data (InputLayer)        [(None, 14922)]      0                                            
__________________________________________________________________________________________________
Images-data (InputLayer)        [(None, 1280)]       0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 1280)         19101440    tf-idf-data[0][0]                
__________________________________________________________________________________________________
tf.math.multiply (TFOpLambda)   (None, 1280)         0           Images-data[0][0]                
                                                                 dense_1[0][0]                 

Create a function to load images from a directory, loops over the images and extracts features using the pretrained model, then compiles with tfidf data into a dataframe.

In [28]:
#Once again, change the path
DATADIR=r"C:\Users\David\Documents\code\Mod_5\5-4-modelling-competition\shopee-product-matching\train_images\\"

#new columns that has the dir + the name of the image
df['Im_dir'] = DATADIR + df['image']


batch_size=500

n_it=int(len(df)/batch_size)
a=0
b=batch_size
batch_list=[]
df2=pd.DataFrame(columns=np.arange(1280))


for i in range(n_it):
    batch_list.append(df[a:b])
    a+=batch_size
    b+=batch_size
batch_list.append(df[a:])


for i in tqdm(range(len(batch_list))):
    df_batch=batch_list[i]
    #the size of the image

    #Empty list 
    data_im=[]

    IM_SIZE=224
    COLOR_MODE='rgb' 
    #the directory for all individual images
    images=df_batch.Im_dir.values

    #loop over batch of the images 
    for im in (images):


        img_array=tf.keras.preprocessing.image.load_img(im,target_size=(IM_SIZE,IM_SIZE),color_mode=COLOR_MODE)
        img_array = tf.keras.preprocessing.image.img_to_array(img_array)
        

        #normalizing
        img_array=img_array/255.0

        #appends the data in the list
        data_im.append(img_array)
    data_im_arr=np.array(data_im)

    ###################################################################
    pretrained_arr=feat_extractor.predict(data_im_arr)

    T=t[batch_list[i].index]

    pred_n=model.predict([pretrained_arr,T])
    df1=pd.DataFrame(pred_n)
    
    df2 = pd.concat([df2,df1],ignore_index=True)

Finally, we will load and fit KNN to cluster our data.

In [29]:
#This is the dataframe we created previously.
X=df2.copy()
neigh = NearestNeighbors(n_neighbors=50) 
neigh.fit(X)
Knn=neigh.kneighbors(X)

We tried normalization as well as a cosine distance model, but found that the data was optimized without it.

In [30]:
#Load the 99th row.
n=99

#Show the distance between first image and closest 'neighbours'.
display(Knn[0][n])
#Show the index of closest 'neighbours'.
display(Knn[1][n])

#Let's compare and see how many neighbours belong to the same label group.
pred=Knn[1][n]
dn=df.loc[df.label_group==df.iloc[n].label_group]
arr=dn.index
print('N=',len(arr))
for p in pred:
    print(p in arr)

distances=Knn[0]

array([0.        , 0.01100821, 0.01101548, 0.01397505, 0.01597449,
       0.01604376, 0.0161487 , 0.01616632, 0.01620424, 0.01623121,
       0.01623779, 0.0163654 , 0.01638376, 0.01639852, 0.0168985 ,
       0.01736714, 0.01750779, 0.01750861, 0.01757717, 0.01766598,
       0.017797  , 0.01785195, 0.01795373, 0.01796868, 0.01801562,
       0.01818861, 0.01828689, 0.0183358 , 0.01841783, 0.01844888,
       0.01846514, 0.01857417, 0.01862657, 0.01865791, 0.01866075,
       0.01875518, 0.01885724, 0.01920053, 0.01920145, 0.01920965,
       0.01923523, 0.01940035, 0.01947386, 0.01948906, 0.01949422,
       0.01952677, 0.01956095, 0.01969019, 0.01975629, 0.01981006])

array([   99,  8668, 27075,    63,  2608, 33322,    75, 12706, 17811,
       14491,  2517, 32578,  7526, 26483, 12688, 12393, 21094,  6734,
       24157,  8561, 13476,  3154,  7096, 12932,  2289, 33769, 30531,
       18450, 22170, 18326, 28489, 13473, 25047, 19819, 10023, 18358,
       12795, 27785, 32423,   849, 21204, 24367, 32454, 24093, 20627,
       28328, 17939, 18520, 10256, 33779], dtype=int64)

N= 41
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False
True
False
False
True
False
False
True
True
False
False
False
True
True
True
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False


Prepare the submission file.

In [31]:
tmp = df.groupby(['label_group'])['posting_id'].unique().to_dict()
df['matches'] = df['label_group'].map(tmp)
df['matches'] = df['matches'].apply(lambda x: ' '.join(x))

This next function will run row-to-row to create an inclusive f1 score.

In [32]:
def f1_score(y_true, y_pred):
    y_true = y_true.apply(lambda x: set(x.split()))
    y_pred = y_pred.apply(lambda x: set(x.split()))
    intersection = np.array([len(x[0] & x[1]) for x in zip(y_true, y_pred)])
    len_y_pred = y_pred.apply(lambda x: len(x)).values
    len_y_true = y_true.apply(lambda x: len(x)).values
    f1 = 2 * intersection / (len_y_pred + len_y_true)
    return f1

In [37]:
predictions = []
threshold=0.015

indices=Knn[1]
for k in range(df2.shape[0]):
    idx = np.where(distances[k,] < threshold)[0]
    ids = indices[k,idx]
    posting_ids = ' '.join(df['posting_id'].iloc[ids].values)
    predictions.append(posting_ids)
df['pred_matches'] = predictions
df['f1'] = f1_score(df['matches'], df['pred_matches'])
score = df['f1'].mean()
score

0.6448471839952333

In [34]:
    distances=Knn[0]

    thresholds=np.arange(0.2,0.5,0.01)
    s=[]
    for t in tqdm(thresholds):
        predictions = []
        threshold=t
        indices=Knn[1]
        for k in range(df2.shape[0]):
            idx = np.where(distances[k,] < threshold)[0]
            ids = indices[k,idx]
            posting_ids = ' '.join(df['posting_id'].iloc[ids].values)
            predictions.append(posting_ids)
        df['pred_matches'] = predictions
        df['f1'] = f1_score(df['matches'], df['pred_matches'])
        score = df['f1'].mean()
        s.append(score)
    s=np.array(s)
    thresholds[np.argmax(s)]


0.2

Now to display the submission file.

In [ ]:
df[['posting_id','pred_matches']]

In [ ]:
df[['posting_id','matches']]